# مقدمه‌ای بر احتمال و آمار
در این دفترچه، با برخی از مفاهیمی که قبلاً مورد بحث قرار داده‌ایم بازی خواهیم کرد. بسیاری از مفاهیم احتمال و آمار به‌خوبی در کتابخانه‌های اصلی پردازش داده در پایتون مانند `numpy` و `pandas` نمایان شده‌اند.


In [ ]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

## متغیرهای تصادفی و توزیع‌ها
بیایید با کشیدن یک نمونه ۳۰ مقداری از توزیع یکنواخت از ۰ تا ۹ شروع کنیم. همچنین میانگین و واریانس را محاسبه خواهیم کرد.


In [ ]:
sample = [ random.randint(0,10) for _ in range(30) ]
print(f"Sample: {sample}")
print(f"Mean = {np.mean(sample)}")
print(f"Variance = {np.var(sample)}")

برای تخمین بصری اینکه چند مقدار مختلف در نمونه وجود دارد، می‌توانیم **هیستوگرام** را رسم کنیم:


In [ ]:
plt.hist(sample)
plt.show()

## تجزیه و تحلیل داده‌های واقعی

میانگین و واریانس در تحلیل داده‌های دنیای واقعی بسیار مهم هستند. بیایید داده‌های مربوط به بازیکنان بیسبال را از [SOCR MLB Height/Weight Data](http://wiki.stat.ucla.edu/socr/index.php/SOCR_Data_MLB_HeightsWeights) بارگذاری کنیم.


In [ ]:
df = pd.read_csv("../../data/SOCR_MLB.tsv",sep='\t', header=None, names=['Name','Team','Role','Weight','Height','Age'])
df


> ما در اینجا از بسته‌ای به نام [**Pandas**](https://pandas.pydata.org/) برای تحلیل داده‌ها استفاده می‌کنیم. در ادامه این دوره، بیشتر درباره Pandas و کار با داده‌ها در پایتون صحبت خواهیم کرد.

بیایید میانگین مقادیر سن، قد و وزن را محاسبه کنیم:


In [ ]:
df[['Age','Height','Weight']].mean()

حالا بیایید روی قد تمرکز کنیم و انحراف معیار و واریانس را محاسبه کنیم:


In [ ]:
print(list(df['Height'])[:20])

In [ ]:
mean = df['Height'].mean()
var = df['Height'].var()
std = df['Height'].std()
print(f"Mean = {mean}\nVariance = {var}\nStandard Deviation = {std}")

علاوه بر میانگین، منطقی است که به مقدار میانه و چارک‌ها نیز نگاه کنیم. آنها می‌توانند با استفاده از **نمودار جعبه‌ای** نمایش داده شوند:


In [ ]:
plt.figure(figsize=(10,2))
plt.boxplot(df['Height'].ffill(), vert=False, showmeans=True)
plt.grid(color='gray', linestyle='dotted')
plt.tight_layout()
plt.show()

ما همچنین می‌توانیم نمودارهای جعبه‌ای از زیرمجموعه‌های مجموعه داده خود بسازیم، به عنوان مثال، گروه‌بندی شده بر اساس نقش بازیکن.


In [ ]:
df.boxplot(column='Height', by='Role', figsize=(10,8))
plt.xticks(rotation='vertical')
plt.tight_layout()
plt.show()

> **توجه**: این نمودار نشان می‌دهد که به طور میانگین، قد بازیکنان پست اول بلندتر از قد بازیکنان پست دوم است. بعداً یاد می‌گیریم که چگونه می‌توانیم این فرضیه را به صورت رسمی‌تر آزمایش کنیم و چگونه نشان دهیم داده‌های ما از نظر آماری معنادار هستند تا این موضوع را اثبات کنیم.

سن، قد و وزن همگی متغیرهای تصادفی پیوسته هستند. به نظر شما توزیع آنها چیست؟ یک راه خوب برای فهمیدن، رسم هیستوگرام مقادیر است:


In [ ]:
df['Weight'].hist(bins=15, figsize=(10,6))
plt.suptitle('Weight distribution of MLB Players')
plt.xlabel('Weight')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

## توزیع نرمال

بیایید یک نمونه ساختگی از وزن‌ها ایجاد کنیم که توزیع نرمالی با همان میانگین و واریانس داده‌های واقعی ما داشته باشد:


In [ ]:
generated = np.random.normal(mean, std, 1000)
generated[:20]

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(generated, bins=15)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(np.random.normal(0,1,50000), bins=300)
plt.tight_layout()
plt.show()

از آنجایی که اکثر مقادیر در دنیای واقعی به طور نرمال توزیع می‌شوند، نباید از یک تولیدکننده اعداد تصادفی یکنواخت برای تولید داده‌های نمونه استفاده کنیم. در اینجا آنچه اتفاق می‌افتد اگر سعی کنیم وزن‌ها را با توزیع یکنواخت (تولید شده توسط `np.random.rand`) ایجاد کنیم، آورده شده است:


In [ ]:
wrong_sample = np.random.rand(1000)*2*std+mean-std
plt.figure(figsize=(10,6))
plt.hist(wrong_sample)
plt.tight_layout()
plt.show()

## بازه‌های اطمینان

حال بیایید بازه‌های اطمینان را برای وزن‌ها و قدهای بازیکنان بیسبال محاسبه کنیم. ما از کد [در این بحث استک‌اورفلو](https://stackoverflow.com/questions/15033511/compute-a-confidence-interval-from-sample-data) استفاده خواهیم کرد:


In [ ]:
import scipy.stats

def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, h

for p in [0.85, 0.9, 0.95]:
    m, h = mean_confidence_interval(df['Weight'].fillna(method='pad'),p)
    print(f"p={p:.2f}, mean = {m:.2f} ± {h:.2f}")

## آزمون فرضیه

بیایید نقش‌های مختلف در مجموعه داده‌های بازیکنان بیسبال خود را بررسی کنیم:


In [ ]:
df.groupby('Role').agg({ 'Weight' : 'mean', 'Height' : 'mean', 'Age' : 'count'}).rename(columns={ 'Age' : 'Count'})

بیایید فرضیه‌ای را بررسی کنیم که بازیکنان اول بیش از بازیکنان دوم قد بلندتر هستند. ساده‌ترین راه برای این کار تست فواصل اطمینان است:


In [ ]:
for p in [0.85,0.9,0.95]:
    m1, h1 = mean_confidence_interval(df.loc[df['Role']=='First_Baseman',['Height']],p)
    m2, h2 = mean_confidence_interval(df.loc[df['Role']=='Second_Baseman',['Height']],p)
    print(f'Conf={p:.2f}, 1st basemen height: {m1-h1[0]:.2f}..{m1+h1[0]:.2f}, 2nd basemen height: {m2-h2[0]:.2f}..{m2+h2[0]:.2f}')

می‌توانیم ببینیم که بازه‌ها همپوشانی ندارند.

یک روش آماری صحیح‌تر برای اثبات فرضیه استفاده از **آزمون t دانشجو** است:


In [ ]:
from scipy.stats import ttest_ind

tval, pval = ttest_ind(df.loc[df['Role']=='First_Baseman',['Height']], df.loc[df['Role']=='Second_Baseman',['Height']],equal_var=False)
print(f"T-value = {tval[0]:.2f}\nP-value: {pval[0]}")

دو مقداری که توسط تابع `ttest_ind` بازگردانده می‌شوند عبارتند از:
* مقدار p-value را می‌توان به عنوان احتمال برابری میانگین دو توزیع در نظر گرفت. در مورد ما، این مقدار بسیار کم است، که به این معنی است که شواهد قوی‌ای وجود دارد که حمایت می‌کند اولی‌ها قد بلندتری دارند.
* مقدار t-value مقدار میانی اختلاف میانگین نرمال‌شده است که در آزمون t استفاده می‌شود و با یک مقدار آستانه برای یک مقدار اطمینان معین مقایسه می‌گردد.


## شبیه‌سازی توزیع نرمال با قضیه حد مرکزی

مولد شبه‌تصادفی در پایتون برای ارائه توزیع یکنواخت طراحی شده است. اگر بخواهیم یک مولد برای توزیع نرمال ایجاد کنیم، می‌توانیم از قضیه حد مرکزی استفاده کنیم. برای به‌دست آوردن یک مقدار با توزیع نرمال، تنها کافی است میانگین نمونه‌ای که از توزیع یکنواخت تولید شده است را محاسبه کنیم.


In [ ]:
def normal_random(sample_size=100):
    sample = [random.uniform(0,1) for _ in range(sample_size) ]
    return sum(sample)/sample_size

sample = [normal_random() for _ in range(100)]
plt.figure(figsize=(10,6))
plt.hist(sample)
plt.tight_layout()
plt.show()

## همبستگی و شرکت بیسبال شرور

همبستگی به ما اجازه می‌دهد روابط بین دنباله‌های داده را پیدا کنیم. در مثال اسباب‌بازی ما، فرض کنیم یک شرکت بیسبال شرور وجود دارد که به بازیکنانش بر اساس قدشان پول پرداخت می‌کند - هر چقدر بازیکن بلندقدتر باشد، پول بیشتری می‌گیرد. فرض کنید حقوق پایه ۱۰۰۰ دلار است و یک پاداش اضافی از ۰ تا ۱۰۰ دلار بسته به قد وجود دارد. بازیکنان واقعی MLB را می‌گیریم و حقوق خیالی آن‌ها را محاسبه می‌کنیم:


In [ ]:
heights = df['Height'].fillna(method='pad')
salaries = 1000+(heights-heights.min())/(heights.max()-heights.mean())*100
print(list(zip(heights, salaries))[:10])

حالا بیایید کوواریانس و هم‌بستگی آن دنباله‌ها را محاسبه کنیم. `np.cov` به ما ماتریس کوواریانس می‌دهد، که یک تعمیم کوواریانس به چند متغیر است. عنصر $M_{ij}$ از ماتریس کوواریانس $M$ هم‌بستگی بین متغیرهای ورودی $X_i$ و $X_j$ است، و مقادیر قطری $M_{ii}$ واریانس $X_{i}$ می‌باشند. به‌طور مشابه، `np.corrcoef` به ما ماتریس هم‌بستگی می‌دهد.


In [ ]:
print(f"Covariance matrix:\n{np.cov(heights, salaries)}")
print(f"Covariance = {np.cov(heights, salaries)[0,1]}")
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

همبستگی برابر با ۱ به این معنی است که یک **رابطه خطی** قوی بین دو متغیر وجود دارد. می‌توانیم رابطه خطی را به صورت بصری با رسم یکی از مقادیر در مقابل دیگری مشاهده کنیم:


In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(heights,salaries)
plt.tight_layout()
plt.show()

بیایید ببینیم چه اتفاقی می‌افتد اگر رابطه خطی نباشد. فرض کنید شرکت ما تصمیم گرفته است وابستگی خطی آشکار بین قدها و حقوق‌ها را پنهان کند و مقداری غیرخطی بودن، مانند `sin`، به فرمول اضافه کند:


In [ ]:
salaries = 1000+np.sin((heights-heights.min())/(heights.max()-heights.mean()))*100
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

در این مورد، همبستگی کمی کمتر است، اما هنوز نسبتاً بالا است. حالا، برای این که رابطه کمتر آشکار باشد، ممکن است بخواهیم با افزودن یک متغیر تصادفی به حقوق، مقداری تصادفی بودن اضافی ایجاد کنیم. بیایید ببینیم چه اتفاقی می‌افتد:


In [ ]:
salaries = 1000+np.sin((heights-heights.min())/(heights.max()-heights.mean()))*100+np.random.random(size=len(heights))*20-10
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(heights, salaries)
plt.tight_layout()
plt.show()

> آیا می‌توانید حدس بزنید چرا نقاط به این شکل در خطوط عمودی قرار گرفته‌اند؟

ما همبستگی بین یک مفهوم مصنوعی مهندسی‌شده مانند حقوق و متغیر مشاهده‌شده *قد* را مشاهده کرده‌ایم. بیایید ببینیم آیا دو متغیر مشاهده‌شده، مانند قد و وزن، نیز با یکدیگر همبسته هستند یا خیر:


In [ ]:
np.corrcoef(df['Height'].ffill(),df['Weight'])

متأسفانه، ما هیچ نتیجه‌ای نگرفتیم - فقط چند مقدار عجیب `nan` وجود داشت. این به این دلیل است که برخی از مقادیر در سری ما تعریف نشده‌اند، که به صورت `nan` نمایش داده می‌شوند، و این باعث می‌شود نتیجه عملیات نیز تعریف نشده باشد. با نگاه به ماتریس می‌توان دید که ستون `Weight` مشکل‌ساز است، زیرا همبستگی خودی بین مقادیر `Height` محاسبه شده است.

> این مثال اهمیت **آماده‌سازی داده‌ها** و **پاک‌سازی** را نشان می‌دهد. بدون داده‌های مناسب نمی‌توانیم هیچ چیزی محاسبه کنیم.

بیایید از متد `fillna` استفاده کنیم تا مقادیر گمشده را پر کنیم و همبستگی را محاسبه کنیم: 


In [ ]:
np.corrcoef(df['Height'].fillna(method='pad'), df['Weight'])

در واقع همبستگی وجود دارد، اما آنقدر قوی نیست که در مثال مصنوعی ما دیده شود. در حقیقت، اگر به نمودار پراکندگی یک مقدار نسبت به مقدار دیگر نگاه کنیم، رابطه بسیار کمتر واضح خواهد بود:


In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(df['Weight'],df['Height'])
plt.xlabel('Weight')
plt.ylabel('Height')
plt.tight_layout()
plt.show()

## نتیجه‌گیری

در این نوت‌بوک یاد گرفتیم چگونه عملیات پایه‌ای روی داده‌ها انجام داده و توابع آماری را محاسبه کنیم. اکنون می‌دانیم چگونه از مجموعه‌ای قوی از ریاضیات و آمار برای اثبات برخی فرضیات استفاده کنیم و چگونه بازه‌های اطمینان را برای متغیرهای دلخواه با توجه به نمونه داده محاسبه کنیم.


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**اعلان مسئولیت**:  
این سند با استفاده از خدمات ترجمه هوش مصنوعی [Co-op Translator](https://github.com/Azure/co-op-translator) ترجمه شده است. در حالی که ما تلاش داریم دقت را حفظ کنیم، لطفاً توجه داشته باشید که ترجمه‌های خودکار ممکن است حاوی خطا یا نادرستی باشند. سند اصلی به زبان بومی آن باید به‌عنوان منبع معتبر در نظر گرفته شود. برای اطلاعات حیاتی، استفاده از ترجمه حرفه‌ای انسانی توصیه می‌شود. ما مسئول هیچ گونه سوءتفاهم یا برداشت نادرستی که از استفاده این ترجمه نشأت گیرد، نیستیم.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
